<a href="https://colab.research.google.com/github/MitjaGo/BAR/blob/main/BAR_beta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="clear: both;">
<div>
      <h2><b>EXPORT</b>4PHOBS</h2>
     <img src="https://scontent.flju1-1.fna.fbcdn.net/v/t39.30808-1/538848259_1373858921407395_3734947029332493437_n.jpg?stp=c221.221.1034.1034a_dst-jpg_s480x480_tt6&_nc_cat=110&ccb=1-7&_nc_sid=2d3e12&_nc_ohc=nLyDiOB5EM8Q7kNvwEw-9Xf&_nc_oc=Adm0RsJmASURcGiXVvWUyJRIrypsmZsQEM5WYIv2uG8ZFEp5f1wteeqopUsLj6bPgFk&_nc_zt=24&_nc_ht=scontent.flju1-1.fna&_nc_gid=J2ABA-OjzFAu8lGZyNEJ3g&oh=00_AffBA_KZ8YxYLhFZ2GHdTmTHnUp2bG6tZWV73qMxnj66Qg&oe=68FAB123" width="180" alt="">
  </div>
  </div>
  <div>
    <p>Generator za BAR cenike</p>
  </div>
</div>



In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML
import base64

output = widgets.Output()

# --- Function to prepare PHOBS dataframe ---
def prepare_phobs_csv(df, hotel_id, los_code, hotel_name):
    df['BAR'] = df['BAR'].apply(lambda x: f"BAR{x}")
    df[hotel_name] = hotel_id
    df['nicla'] = 0
    df['LOS'] = f"YIELD{los_code}"
    cols = [hotel_name, 'Datum', 'nicla', 'BAR', 'LOS']
    df = df.reindex(columns=cols)
    df.columns = [hotel_name, 'Datum', 'nicla', 'BAR', 'Yield']
    return df

# --- Function to create a download button with timed confirmation ---
def create_download_button(df, filename, description):
    button = widgets.Button(description=description, button_style='info', layout=widgets.Layout(width="33%"))

    def on_click(b):
        with output:
            output.clear_output()
            display(HTML(f"<b id='download_msg'>{description}</b>: Your CSV is downloading..."))
            csv_bytes = df.to_csv(index=False, header=False).encode()
            b64 = base64.b64encode(csv_bytes).decode()

            # JavaScript: auto-download + remove message after 3 seconds
            js = f"""
            <a id="download_link" download="{filename}.csv" href="data:text/csv;base64,{b64}"></a>
            <script>
                document.getElementById('download_link').click();
                setTimeout(function() {{
                    var msg = document.getElementById('download_msg');
                    if(msg) {{ msg.remove(); }}
                }}, 3000);
            </script>
            """
            display(HTML(js))

    button.on_click(on_click)
    return button

# --- Define Google Sheet IDs and sheets ---
hotels = [
    ("ApartmajiAdria", 1001011, 16283),
    ("StandardMobileHomes", 1001012, 16552),
    ("PremiumMobileHomes", 1001011, 16242),
    ("HotelConvent", 1001011, 16355),
    ("OliveSuites", 1001011, 16244),
    ("Vilebrezbalkona", 1001011, 16284),
    ("Vilezbalkonom", 1001011, 15985)
]

gsheet_id = "15HJ7wxyUmo-gcl5_y1M9gl4Ti-JSsYEJZCjoI76s-Xk"

# --- Load data and create buttons ---
buttons = []
for hotel_name, hotel_id, los_code in hotels:
    sheet_url = f"https://docs.google.com/spreadsheets/d/{gsheet_id}/gviz/tq?tqx=out:csv&sheet={hotel_name}"
    df = pd.read_csv(sheet_url)
    df = prepare_phobs_csv(df, hotel_id, los_code, hotel_name)
    button = create_download_button(df, f"{hotel_name}-4-Phobs", hotel_name.replace("_", " "))
    buttons.append(button)

# --- Layout buttons ---
h1 = widgets.HBox(buttons[0:3])
h2 = widgets.HBox(buttons[3:6])
h3 = widgets.HBox(buttons[6:])
vbox = widgets.VBox([h1, h2, h3])

display(vbox, output)
